In [21]:
import numpy as np
import json
import os
import tensorflow as tf
import onnx

In [22]:
# Save tensor as binary file
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


In [23]:
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    intermediate_outputs = {}  # To store outputs for sequential execution

    # Run input through the model layer by layer
    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor to file
        save_tensor_as_bin(x, f"{save_dir}/input/{layer_name}_input.bin")

        # Save weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                save_tensor_as_bin(weight.numpy(), f"{save_dir}/weights/{layer_name}_{weight_name}.bin")

        # Compute layer output
        x = layer(x)

        # Save output tensor to file
        save_tensor_as_bin(x, f"{save_dir}/output/{layer_name}_output.bin")

        # Save layer attributes
        attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
        }
        if hasattr(layer, 'kernel_size'):
            attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            attributes["activation"] = layer.activation.__name__

        save_json(attributes, f"{save_dir}/reference/{layer_name}_attributes.json")

        # Store intermediate outputs for sequential execution
        intermediate_outputs[layer_name] = x.numpy()

    return intermediate_outputs


In [24]:
model = tf.keras.models.load_model("/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/CPU_trained_models/cifar10_model_4.h5")

In [25]:
input_tensor = tf.random.normal([1, 32, 32, 3])  # Random input for CIFAR-10
layer_wise_dump_tf(model, input_tensor, save_dir="data")

{'conv2d': array([[[[0.        , 0.5187326 , 0.        , ..., 0.5454693 ,
           0.        , 0.4002128 ],
          [0.        , 1.1369877 , 0.        , ..., 0.39820287,
           0.        , 0.        ],
          [0.        , 1.6051784 , 0.47569162, ..., 0.5553771 ,
           0.        , 0.        ],
          ...,
          [0.        , 0.8535037 , 0.07060243, ..., 0.61017597,
           0.        , 0.24025297],
          [0.        , 0.        , 0.        , ..., 0.47379935,
           0.28880492, 0.04708022],
          [0.        , 0.        , 1.2007746 , ..., 0.06606665,
           0.        , 0.2532372 ]],
 
         [[0.27656144, 0.        , 0.        , ..., 1.6475273 ,
           0.        , 0.18196492],
          [0.        , 0.20066035, 0.20646815, ..., 2.1185465 ,
           0.29523885, 0.1874559 ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.03672086],
          ...,
          [0.        , 0.        , 0.78143877, ..., 0.736